<a href="https://colab.research.google.com/github/ulung3ko/text-anaytics-assignment-2-topic-modelling/blob/main/topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identifikasi Masalah
## Latar Belakang
Dalam dunia akademik, artikel ilmiah diterbitkan setiap hari. Volume informasi ini menyulitkan akademisi baik itu mahasiswa, dosen, dan peneliti untuk mengikuti perkembangan terbaru secara menyeluruh. Disinilah penting nya topic modeling, dengan teknik ini, kita dapat secara otomatis mengelompokkan kumpulan abstrak artikel ilmiah ke dalam topik-topik utama seperti Artificial Intelligence, Cybersecurity, atau Computer Vision.

Topic modeling memungkinkan kita untuk:

- Mengidentifikasi tren penelitian yang sedang berkembang.
- Menemukan artikel relevan tanpa membaca satu per satu.
- Menyederhanakan eksplorasi literatur dalam jumlah besar.

Dalam tugas ini, kita akan membandingkan dua metode topic modeling:

- Latent Dirichlet Allocation (LDA).
- BERTopic.

Fokus tugas ini yaitu menilai model mana yang memberikan hasil pengelompokan topik yang lebih relevan, dapat dimengerti, dan sesuai dengan kategori asli artikel.

## Pertanyaan Penelitian
1. Topik apa saja yang berhasil ditemukan oleh masing-masing metode?

2. Berapa jumlah topik optimal, dan model mana yang memberikan skor kualitas topik terbaik?

3. Seberapa sesuai hasil topik dari masing-masing metode dengan label kategori asli dari artikel?



# Pengumpulan Dataset
## Sumber Data
Untuk tugas topic modeling ini, dataset yang digunakan adalah dataset publik "ArXiv Dataset" yang tersedia di platform Kaggle. Dataset ini berisi metadata dari jutaan publikasi ilmiah dari penerbit arXiv.org.

Setiap entri mencakup informasi seperti:
- id: ID unik artikel
- title: Judul artikel
- abstract: Ringkasan isi artikel (digunakan sebagai teks utama analisis)
- categories: Label bidang ilmu (misal: cs.AI, cs.CV, math.CO) yang akan digunakan untuk evaluasi model
- update_date: Tanggal terakhir artikel diperbarui

Dataset ini cocok dengan implementasi topic modeling karena berisi abstrak berkualitas tinggi, terstruktur, dan kaya informasi—ideal untuk pemrosesan bahasa alami (NLP).

Link Kaggle : https://www.kaggle.com/datasets/Cornell-University/arxiv

## Pengumpulan Data
Dataset asli berukuran besar (lebih dari 2 juta entri) dan disimpan dalam format JSON Lines (.jsonl), di mana setiap baris adalah satu objek JSON.

Supaya lebih relevan, dataset yang artikel yang digunakan hanya tahun 2021 keatas. Dari tahap ini, diperoleh DataFrame akhir (df_recent) dengan 1.055.586 baris

In [1]:
# Ini merupakan kode dari kaggle sendiri untuk mendownload dataset nya
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

100%|██████████| 1.46G/1.46G [00:38<00:00, 40.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/239


In [2]:
"""Kode ini digunakan untuk memindahkan file dataset yang telah ter download
kedalam direktori '/content/dataset' google colab"""
import shutil
# Source folder dari kagglehub
src_path = "/root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/239"

# Target folder (direktori kerja biasa)
dst_path = "/content/dataset"

# Salin semua isi folder
shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

print("Dataset telah dipindahkan ke:", dst_path)

Dataset telah dipindahkan ke: /content/dataset


In [4]:
"""Kode dibawah digunakan untuk mengkonversi file dataset yang berformat json
ke dalam pandas dataframe, dan saat proses nya difilter hanya artikel atau paper
dengan tahun >= 2021"""

import pandas as pd
import json

data = []
file_path = '/content/dataset/arxiv-metadata-oai-snapshot.json'
start_year = 2021

with open(file_path, 'r') as f:
    for line in f:
        # Megubah setiap baris menjadi dictionary
        parsed_line = json.loads(line)

        # Membaca tahun dari kolom 'update_date'
        # try-except digunakan untuk antisipasi mana tau ada data yang aneh
        try:
            # Mendapatkan tahun dari kolom 'update date'
            year = int(parsed_line['update_date'][:4])

            # Simpa data >= 2021
            if year >= start_year:
                data.append(parsed_line)
        except (ValueError, TypeError):
            # Jika ada error maka dilanjutkan proses nya
            continue

# Membuat dataframe dari data-data yang sudah difilter
df = pd.DataFrame(data)

# Cek hasil dataframe
if not df.empty:
    print(f"Berhasil membaca {len(df)} baris data dari tahun {start_year} ke atas.")
    print("\nInformasi DataFrame:")
    df.info()

    print("\n5 baris pertama data terbaru:")
    print(df.head())
else:
    print(f"Tidak ada data yang ditemukan dari tahun {start_year} ke atas.")

Berhasil membaca 1055586 baris data dari tahun 2021 ke atas.

Informasi DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055586 entries, 0 to 1055585
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1055586 non-null  object
 1   submitter       1055554 non-null  object
 2   authors         1055586 non-null  object
 3   title           1055586 non-null  object
 4   comments        673326 non-null   object
 5   journal-ref     208190 non-null   object
 6   doi             317727 non-null   object
 7   report-no       25292 non-null    object
 8   categories      1055586 non-null  object
 9   license         1054272 non-null  object
 10  abstract        1055586 non-null  object
 11  versions        1055586 non-null  object
 12  update_date     1055586 non-null  object
 13  authors_parsed  1055586 non-null  object
dtypes: object(14)
memory usage: 112.7+ MB

5 baris pertama data ter